### Abstract

In this notebook, I would try to implement the random-forest classification algorithm. 

The implementation is largely inspired by a post from [machinelearningmastery.com](https://machinelearningmastery.com/implement-random-forest-scratch-python/), and reconstructed with a few improvements: 

- Use the efficient libraries such as *Pandas* and *Numpy*, instead of basic *list* and *dictionary*.


In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

### Dataset

Here we use [a dataset from UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks%29) for testing.

In [4]:
!ls sonar*

sonar.all-data	sonar.mines  sonar.names  sonar.rocks


In [8]:
! head sonar.all-data -n 3

0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.

In [28]:
df_data = pd.read_csv('./sonar.all-data', header=None)

print('Num of rows:', len(df_data))
print('Num of columns:', len(df_data.columns))

Num of rows: 208
Num of columns: 61


In [148]:
def tree_node_split(df, column_index, split_value):
    """
        Split the dataframe based on the values within a specified column,
          the ones (with rows) < 'split_value' on the left subnode,
          and the one >= 'split_value' on the right subnode.  
        
        df: DataFrame
        
        return:  two dataframes
    """
    left_node = []
    right_node = []
    
    for row_index, row in df.iterrows():
        if row[column_index] < split_value:
            left_node.append(df.iloc[row_index])
        else:
            right_node.append(df.iloc[row_index])
    
    split_groups = [left_node, right_node]
    split_groups = [pd.concat(group, axis=1).T if len(group) else pd.DataFrame() for group in split_groups]
    return split_groups
    #return (pd.concat(left_node, axis=1).T, pd.concat(right_node, axis=1).T)


def gini_index(df_groups, target_feature_index):
    """
    """
    total_samples = sum([len(group) for group in df_groups])
    gini_value = 0.0
    
    for group in df_groups:
        group_size = float(len(group))
        if group_size == 0:
            continue
        
        class_count = group[target_feature_index].value_counts()
        group_gini = sum([(count/group_size) **2 for count in class_count])
        
        # weight the group gini value by its relative size
        gini_value += (1 - group_gini) * (group_size / total_samples)
    
    return gini_value


def get_best_split(df, n_split_features, target_feature_index):
    """
        
    """
    import random
    import sys
    feature_indice = list(set(range(len(df.columns))) - set([target_feature_index]))
    sample_features = random.sample(feature_indice, n_split_features)

    b_feature_index = None
    b_feature_value = None
    b_gini_index = sys.maxsize
    b_split_groups = None
    # split over each feature, on each of the unique feature values
    for feature_index in sample_features:
        split_values = df[feature_index].unique()
        
        for feature_value in split_values:
            split_groups = tree_node_split(df, feature_index, feature_value)
            gini_value = gini_index(split_groups, target_feature_index)

            if gini_value < b_gini_index:
                b_gini_index = gini_value
                b_feature_index = feature_index
                b_feature_value = feature_value
                b_split_groups = split_groups
    
    return {"split_feature": b_feature_index,
            "split_value": b_feature_value,
            "gini_index": b_gini_index,
            "split_groups": b_split_groups}

In [149]:
ret = get_best_split(df_data, 3, target_feature_index=60)

In [96]:
df_data[0].describe()

count    208.000000
mean       0.029164
std        0.022991
min        0.001500
25%        0.013350
50%        0.022800
75%        0.035550
max        0.137100
Name: 0, dtype: float64

In [103]:
median = df_data[0].median()
per_75 = 0.0355
per_25 = 0.0135

group_split = tree_node_split(df_data, column_index=0, split_value=per_25)

left_node, right_node = group_split

print('split with median: {}, length: {}'.format(median, len(left_node)))

split with median: 0.0228, length: 53


In [98]:
left_node.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.02,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.018,0.0084,0.009,0.0032,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.228,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.018,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.01,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.015,0.0085,0.0073,0.005,0.0044,0.004,0.0117,R
5,0.0286,0.0453,0.0277,0.0174,0.0384,0.099,0.1201,0.1833,0.2105,0.3039,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R
6,0.0317,0.0956,0.1321,0.1408,0.1674,0.171,0.0731,0.1401,0.2083,0.3513,...,0.0201,0.0248,0.0131,0.007,0.0138,0.0092,0.0143,0.0036,0.0103,R


In [104]:
gini_index(group_split, target_feature_index=60)

0.48102907439486875

In [90]:
value_count = left_node[60].value_counts()

In [91]:
value_count

R    59
M    44
Name: 60, dtype: int64